In [1]:
## WordCount en PySpark

In [2]:
## Definición del problema
##  Se desea contar la frecuencia de las palabras que aparecen en varios archivos de textos. Para simplificar el problema, pruebe el algoritmo con los archivos generados en 
##  las siguientes celdas.
##  Nota.— Los archivos se encuentran en el directorio wordcount de la carpeta de trabajo.

In [3]:
##
## Se crea el directorio wordcount en la carpeta actual de trabajo
## y se escriben tres archivos en ella.
##
!mkdir -p wordcount/

In [4]:
%%writefile wordcount/text0.txt
Analytics is the discovery, interpretation, and communication of meaningful patterns
in data. Especially valuable in areas rich with recorded information, analytics relies
on the simultaneous application of statistics, computer programming and operations research
to quantify performance.

Organizations may apply analytics to business data to describe, predict, and improve business
performance. Specifically, areas within analytics include predictive analytics, prescriptive
analytics, enterprise decision management, descriptive analytics, cognitive analytics, Big
Data Analytics, retail analytics, store assortment and stock-keeping unit optimization,
marketing optimization and marketing mix modeling, web analytics, call analytics, speech
analytics, sales force sizing and optimization, price and promotion modeling, predictive
science, credit risk analysis, and fraud analytics. Since analytics can require extensive
computation (see big data), the algorithms and software used for analytics harness the most
current methods in computer science, statistics, and mathematics.

Writing wordcount/text0.txt


In [5]:
%%writefile wordcount/text1.txt
The field of data analysis. Analytics often involves studying past historical data to
research potential trends, to analyze the effects of certain decisions or events, or to
evaluate the performance of a given tool or scenario. The goal of analytics is to improve
the business by gaining knowledge which can be used to make improvements or changes.

Writing wordcount/text1.txt


In [6]:
%%writefile wordcount/text2.txt
Data analytics (DA) is the process of examining data sets in order to draw conclusions
about the information they contain, increasingly with the aid of specialized systems
and software. Data analytics technologies and techniques are widely used in commercial
industries to enable organizations to make more-informed business decisions and by
scientists and researchers to verify or disprove scientific models, theories and
hypotheses.

Writing wordcount/text2.txt


In [7]:
## Solución: Algoritmo MapReduce
##   MapReduce es el término utilizado para describir un modelo de programación en paralelo que permite el procesamiento de grandes volúmenes de datos o Big Data 
##   que resultan difíciles de procesar en las aplicaciones tradicionales de procesamiento de datos. En el concepto de Big Data convergen las técnicas de almacenamiento 
##   distribuido de datos con la computación de alto desempeño mediante clusters.

In [8]:
## Apache Spark
##   Spark es un modelo computacional en el cual se elimina elimina a escritura a disco entre jobs (sólo se escribe cuando es necesario), lo que permite que el proceso 
##   se ejecute mucho más rápido. Sus principales componentes son las siguientes:
##      SparkRDD: opera sobre conjuntos de datos distribuidos mediante operaciones MapReduce.
##      SparkQL: Implementación del lenguaje SQL que puede ejecutarse sobre datos estructurados como tablas.
##      SparkML: Implementación de algoritmos de aprendizaje estadístico y aprendizaje automática que operan sobre datos estructurados como tablas.

In [9]:
## Conteo de palabras usando SparkRDD
##  A continuación se realiza el conteo de frecuencia de palabras usando SparkRDD. Se asume que Spark está ejecutándose en un cluster.

In [10]:
##
## Se usan un directorio temporal en el HDFS. La siguiente
## instrucción muestra el contenido del dicho directorio
##
!hdfs dfs -ls /tmp

In [11]:
##
## Crea la carpeta wordcount en el hdfs
##
!hdfs dfs -mkdir /tmp/wordcount

In [12]:
##
## Verifica la creación de la carpeta
##
!hdfs dfs -ls /tmp/

Found 1 items
drwxr-xr-x   - root supergroup          0 2019-11-28 02:19 /tmp/wordcount


In [13]:
##
## Copia los archvios del directorio local wordcount/
## al directorio /tmp/wordcount/ en el hdfs
##
!hdfs dfs -copyFromLocal wordcount/*  /tmp/wordcount/

In [14]:
##
## Verifica que los archivos esten copiados
## en el hdfs
##
!hdfs dfs -ls /tmp/wordcount

Found 3 items
-rw-r--r--   1 root supergroup       1082 2019-11-28 02:19 /tmp/wordcount/text0.txt
-rw-r--r--   1 root supergroup        349 2019-11-28 02:19 /tmp/wordcount/text1.txt
-rw-r--r--   1 root supergroup        435 2019-11-28 02:19 /tmp/wordcount/text2.txt


In [15]:
## Implementación en PySpark

In [16]:
##
## findspark: Permite usar PySpark como una libreria de Python
##
import findspark
findspark.init()

##
## Importa las librerias requeridas para conectar
## a Python con PySpark
##
from pyspark import SparkConf, SparkContext

##
## operador de agregación (MapReduce)
##
from operator import add

##
## Nombre de la aplicación en el cluster
##
APP_NAME = "My Spark Application"

##
## Configure Spark
##
conf = SparkConf().setAppName(APP_NAME)
sc = SparkContext(conf=conf)
sc.setLogLevel('ERROR')

In [17]:
#
## Lee los archivos del hdfs y los carga
## a la variable text
##
text = sc.textFile("/tmp/wordcount/*.txt")

## Se imprimen las primeras 10 líneas
text.collect()[0:10]

['Analytics is the discovery, interpretation, and communication of meaningful patterns',
 'in data. Especially valuable in areas rich with recorded information, analytics relies',
 'on the simultaneous application of statistics, computer programming and operations research',
 'to quantify performance.',
 '',
 'Organizations may apply analytics to business data to describe, predict, and improve business',
 'performance. Specifically, areas within analytics include predictive analytics, prescriptive',
 'analytics, enterprise decision management, descriptive analytics, cognitive analytics, Big',
 'Data Analytics, retail analytics, store assortment and stock-keeping unit optimization,',
 'marketing optimization and marketing mix modeling, web analytics, call analytics, speech']

In [18]:
##
## separa por palabras (split)
## con una palabra por registro
##
words = text.flatMap(lambda x: x.split())

## Se imprimen las primeras 10 palabras
words.collect()[0:10]

['Analytics',
 'is',
 'the',
 'discovery,',
 'interpretation,',
 'and',
 'communication',
 'of',
 'meaningful',
 'patterns']

In [19]:
##
## Genera las parejas <clave, valor> representandolas
## com la tupla (word, 1)
##
wc = words.map(lambda x: (x,1))
wc.collect()[0:10]

[('Analytics', 1),
 ('is', 1),
 ('the', 1),
 ('discovery,', 1),
 ('interpretation,', 1),
 ('and', 1),
 ('communication', 1),
 ('of', 1),
 ('meaningful', 1),
 ('patterns', 1)]

In [20]:
##
## Suma los valores para la misma clave.
## Spark internamente ordena por claves
##
counts = wc.reduceByKey(add)
counts.collect()[0:10]

[('interpretation,', 1),
 ('of', 8),
 ('in', 5),
 ('data.', 1),
 ('Especially', 1),
 ('analytics', 8),
 ('simultaneous', 1),
 ('operations', 1),
 ('research', 2),
 ('quantify', 1)]

In [21]:
##
## Escribe los resultados al directorio `/tmp/output`
##
counts.saveAsTextFile("/tmp/output")

In [22]:
## Archivo de resultados
##   Los resultados son escritos al HDFS.

In [23]:
!hdfs dfs -ls /tmp/

Found 2 items
drwxr-xr-x   - root supergroup          0 2019-11-28 02:21 /tmp/output
drwxr-xr-x   - root supergroup          0 2019-11-28 02:19 /tmp/wordcount


In [24]:
## Archivos con los resultados. Note que se
## generan varios archivos de resultados.
!hdfs dfs -ls /tmp/output/

Found 5 items
-rw-r--r--   1 root supergroup          0 2019-11-28 02:21 /tmp/output/_SUCCESS
-rw-r--r--   1 root supergroup        778 2019-11-28 02:21 /tmp/output/part-00000
-rw-r--r--   1 root supergroup        562 2019-11-28 02:21 /tmp/output/part-00001
-rw-r--r--   1 root supergroup        510 2019-11-28 02:21 /tmp/output/part-00002
-rw-r--r--   1 root supergroup        594 2019-11-28 02:21 /tmp/output/part-00003


In [25]:
## El archivo /tmp/output/_SUCCESS es un archivo vacio que indica que el programa fue ejecutado correctamente.

In [26]:
!hdfs dfs -cat /tmp/output/part-00000

('interpretation,', 1)
('of', 8)
('in', 5)
('data.', 1)
('Especially', 1)
('analytics', 8)
('simultaneous', 1)
('operations', 1)
('research', 2)
('quantify', 1)
('Organizations', 1)
('may', 1)
('business', 4)
('predict,', 1)
('include', 1)
('decision', 1)
('descriptive', 1)
('store', 1)
('optimization,', 2)
('modeling,', 2)
('speech', 1)
('promotion', 1)
('risk', 1)
('fraud', 1)
('Since', 1)
('algorithms', 1)
('used', 3)
('harness', 1)
('current', 1)
('field', 1)
('involves', 1)
('studying', 1)
('potential', 1)
('trends,', 1)
('performance', 1)
('goal', 1)
('changes.', 1)
('process', 1)
('draw', 1)
('specialized', 1)
('systems', 1)
('software.', 1)
('techniques', 1)
('are', 1)
('commercial', 1)
('organizations', 1)
('disprove', 1)
('scientific', 1)
('hypotheses.', 1)


In [27]:
## Movimiento de los archivos de resultados a la máquina local

In [28]:
## crea la carpeta local para poder mover los archivos
!mkdir -p output
!hdfs dfs -copyToLocal /tmp/output/* output/

In [29]:
!ls -l output/*

-rwxrwxrwx 1 1000 1000   0 Nov 28 02:23 output/_SUCCESS
-rwxrwxrwx 1 1000 1000 778 Nov 28 02:23 output/part-00000
-rwxrwxrwx 1 1000 1000 562 Nov 28 02:23 output/part-00001
-rwxrwxrwx 1 1000 1000 510 Nov 28 02:23 output/part-00002
-rwxrwxrwx 1 1000 1000 594 Nov 28 02:23 output/part-00003


In [30]:
#spark.stop()

In [31]:
## Limpieza de las carpetas de trabajo

In [32]:
!rm -rf wordcount
!rm -rf output
!hdfs dfs -rm -r -f /tmp/wordcount/
!hdfs dfs -rm -r -f /tmp/output/

Deleted /tmp/wordcount
Deleted /tmp/output


In [33]:
!hdfs dfs -ls /tmp

In [ ]:
spark.stop()